<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Calling Con2Prim Routines

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the various methods for calling the provided conservative-to-primitive routines.

### Required and recommended citations:

* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).
* **(Recommended)** [Noble, Gammie, McKinney, Del Zanna. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics (2006)](https://arxiv.org/abs/astro-ph/0512420).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [**Introduction**](#introduction)
1. [**Calling Con2Prim routines**](#c2p_routines)
1. [**Provided Con2Prim Methods**](c2p_methods)
    1. [*Hybrid EOS Methods*](#hybrid_eos)
    1. [*Tabulated EOS Methods*](#tabulated_eos)
    1. [*The `Multi_Method` Functions*](#multi_method)
    1. [*The `Select_Method` Functions*](#select_method)

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

GRHayL provides a variety of conservative-to-primitive (C2P) algorithms. These can be directly used, and the `Hybrid_Select_Method` function provides the ability to switch between C2P methods at runtime. The `Hybrid_Multi_Method` function uses `Hybrid_Select_Method` and allows for up to 3 C2P methods to serve as backup methods if the preceeding methods fail.

<a id='c2p_routines'></a>

# Calling Con2Prim routines \[Back to [top](#toc)\]
$$\label{c2p_routines}$$

First, let's look at the argument list which is shared by all conservative-to-primitive routines:

```
int Hybrid_Multi_Method(
      const GRHayL_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      const conservative_quantities *restrict cons,
      const primitive_quantities *restrict prims,
      primitive_quantities *restrict prims_guess,
      con2prim_diagnostics *restrict diagnostics)
```
The first two arguments appear in most GRHayL functions and contain information on the simulation and equation-of-state parameters. The next struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges. The conservative_quantities input contains the **undensitized** conservative variables. Since evolving densitized variables is quite common, we provide the `undensitize_conservatives` function (TODO: link) which takes a struct containing densitized variables and packs a new struct with undensitized quantities to serve as input for this function. The final input is the primitive_quantities struct `prims`, which must at least contain the $B^i$ variables.

The outputted primitives are packed into the primitive_quantities struct `prims_guess`, which is also used as temporary storage for the initial guess for the method. If the parameter `params->calc_prim_guess` is true, then `prims_guess` is set by the `guess_primitives` function. Otherwise, the initial guess is set to whatever was provided via the `prims` input. The $B^i$ in `prims_guess` are always set equal to the $B^i$ in `prims`.

These arguments are identical for all the C2P routines, so switching between individual routine functions is very simple.

<a id='c2p_methods'></a>

# Provided Con2Prim Methods \[Back to [top](#toc)\]
$$\label{c2p_methods}$$

<a id='hybrid_eos'></a>

## Hybrid EOS Methods \[Back to [top](#toc)\]
$$\label{hybrid_eos}$$

Currently, GRHayL supports the following methods for the Hybrid EOS:
- Noble2D
- Noble1D
The Noble routines are detailed in (TODO: link). 

<a id='tabulated_eos'></a>

## Tabulated EOS Methods \[Back to [top](#toc)\]
$$\label{tabulated_eos}$$

Currently, GRHayL supports the following methods for the Tabulated EOS:
- ?
- ?
The ? routines are detailed in (TODO: link).

<a id='multi_method'></a>

## The `Multi_Method` Functions \[Back to [top](#toc)\]
$$\label{multi_method}$$

The `Hybrid_Multi_Method` function has the following structure
```
  set B^i in prims_guess
  set other prims_guess quantities from either prims struct or guess_primitives()
  call params->main_routine C2P function

  if(C2p failed and backup 1 is set)
    set other prims_guess quantities from either prims struct or guess_primitives()
    call params->backup_routine[0] C2P function

    if(C2p failed and backup 2 is set)
      set other prims_guess quantities from either prims struct or guess_primitives()
      call params->backup_routine[1] C2P function

      if(C2p failed and backup 3 is set)
        set other prims_guess quantities from either prims struct or guess_primitives()
        call params->backup_routine[2] C2P function
```
If more backups are desired, the code inside `Hybrid_Multi_Method` can be copied directly into the function calling this one with as many tiered if statements as desired. The parameter in the GRHayL_parameters struct only allows for 3 values, so any additional options will have to be handled in-house by the user code.

TODO: add tabulated_multi_method

<a id='select_method'></a>

## The `Select_Method` Functions \[Back to [top](#toc)\]
$$\label{select_method}$$

The function which calls the C2P routines is `Hybrid_Select_Method`. It differs from all other C2P routines by a single argument
```
int Hybrid_Select_Method(const GRHayL_parameters *restrict params,
                         const eos_parameters *restrict eos,
                         const int c2p_key,
                         const metric_quantities *restrict metric,
                         const conservative_quantities *restrict cons,
                         primitive_quantities *restrict prims,
                         con2prim_diagnostics *restrict diagnostics )
```
This function simply consists of a switch statement that calls the C2P routine matching the additional argument `c2p_key`.

TODO: add tabulated